<a href="https://colab.research.google.com/github/ssheggrud/Mod_20_Project/blob/05_Riley/2005_Contributions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **2005 Indivdual Contributions**

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-2.4.8'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

import pandas as pd


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
import findspark
findspark.add_packages('mysql:mysql-connector-java:8.0.11')

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-14 02:55:05--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.66MB/s    in 0.2s    

2021-09-14 02:55:06 (5.66 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
# Start Spark session
#from pyspark.sql import SparkSession
#from pyspark.sql.functions import col
#spark = SparkSession.builder.appName("FinalProject").getOrCreate()

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/Individual+Contributions+2005+(UF).csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Individual+Contributions+2005+(UF).csv"), sep=",", header=True)
df.show()

+--------+--------+------+--------+-------------+----------+------+---------+------+--------+------+----------+--------+---------------+--------------+--------------------+------+------+------------------+---------+---------+-------------+-----+-----+----------+-------+--------+----------+-------+--------+-----+---------+--------+----------+--------+----------+--------+--------+--------+-------+-----+------+----------+----------+----------+----------+--------+---------+---------+--------+---------+------+-------+----------+
|ELECTION|OFFICECD|CANDID|CANCLASS|     CANDLAST| CANDFIRST|CANDMI|COMMITTEE|FILING|SCHEDULE|PAGENO|SEQUENCENO|   REFNO|           DATE|    REFUNDDATE|                NAME|C_CODE| STRNO|           STRNAME|APARTMENT|BOROUGHCD|         CITY|STATE|  ZIP|OCCUPATION|EMPNAME|EMPSTRNO|EMPSTRNAME|EMPCITY|EMPSTATE| AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|INTERMNO|INTERMNAME|INTSTRNO|INTSTRNM|INTAPTNO|INTCITY|INTST|INTZIP|INTEMPNAME|INTEMPSTNO|INTEMPSTNM|INTEMPCITY|INTEMPST|INTOCC

In [7]:
# Remove multiple columns
list = ['COMMITTEE', 'FILING', 'SCHEDULE', 'PAGENO', 'SEQUENCENO','REFNO', 'STRNO', 'STRNAME', 'APARTMENT', 'OCCUPATION', 'EMPNAME', 'EMPSTRNO', 'EMPSTRNAME',
                'EMPCITY','INTERMNO', 'REFUNDDATE', 'INTERMNAME', 'INTSTRNO', 'INTSTRNM', 'INTSTRNM', 'INTAPTNO',
                'INTCITY', 'INTST','INTZIP', 'INTEMPNAME', 'INTEMPSTNO', 'INTEMPSTNM', 'INTEMPCITY', 'INTEMPST', 'INTOCCUPA' ,'PURPOSECD', 'EXEMPTCD','ADJTYPECD', 'RR_IND', 'SEG_IND','INT_C_CODE'] 
df = df.drop(*list)
df.show()

+--------+--------+------+--------+-------------+----------+------+---------------+--------------------+------+---------+-------------+-----+-----+--------+-----+---------+--------+----------+
|ELECTION|OFFICECD|CANDID|CANCLASS|     CANDLAST| CANDFIRST|CANDMI|           DATE|                NAME|C_CODE|BOROUGHCD|         CITY|STATE|  ZIP|EMPSTATE| AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|
+--------+--------+------+--------+-------------+----------+------+---------------+--------------------+------+---------+-------------+-----+-----+--------+-----+---------+--------+----------+
|    2005|       5|   863|       P|        Samad|    Maryam|     A| 5/13/2005 0:00|(Head) Coach, Big...|   IND|        K|     Brooklyn|   NY|11212|    null|   10|       10|       0|         1|
|    2005|       4|    E9|       P|     Marshall|     Helen|     M| 4/27/2004 0:00|     005 New Gen LLC|   LLC|        Q|       Queens|   NY|11354|    null|  500|        0|       0|         2|
|    2005|       5|   758|       P|

In [8]:
# check data types
df.dtypes

[('ELECTION', 'string'),
 ('OFFICECD', 'string'),
 ('CANDID', 'string'),
 ('CANCLASS', 'string'),
 ('CANDLAST', 'string'),
 ('CANDFIRST', 'string'),
 ('CANDMI', 'string'),
 ('DATE', 'string'),
 ('NAME', 'string'),
 ('C_CODE', 'string'),
 ('BOROUGHCD', 'string'),
 ('CITY', 'string'),
 ('STATE', 'string'),
 ('ZIP', 'string'),
 ('EMPSTATE', 'string'),
 ('AMNT', 'string'),
 ('MATCHAMNT', 'string'),
 ('PREVAMNT', 'string'),
 ('PAY_METHOD', 'string')]

In [9]:
#Change column names
df1 = df \
.withColumnRenamed("ELECTION", "Election") \
.withColumnRenamed("RECIPID", "CandidateID") \
.withColumnRenamed("RECIPNAME", "CandidateName") \
.withColumnRenamed("DATE", "Date") \
.withColumnRenamed("NAME", "Name") \
.withColumnRenamed("C_CODE", "ContributerType") \
.withColumnRenamed("BOROUGHCD", "BoroughName") \
.withColumnRenamed("CITY", "City") \
.withColumnRenamed("STATE", "State") \
.withColumnRenamed("ZIP", "ZipCode") \
.withColumnRenamed("EMPSTATE", "ContributionState") \
.withColumnRenamed("AMNT", "Amount") \
.withColumnRenamed("MATCHAMNT", "MatchAmt") \
.withColumnRenamed("PREVAMNT", "PrevAmt") \
.withColumnRenamed("PAY_METHOD", "PayMethod")


df1.printSchema()

root
 |-- Election: string (nullable = true)
 |-- OFFICECD: string (nullable = true)
 |-- CANDID: string (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CANDLAST: string (nullable = true)
 |-- CANDFIRST: string (nullable = true)
 |-- CANDMI: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- ContributerType: string (nullable = true)
 |-- BoroughName: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: string (nullable = true)
 |-- ContributionState: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- MatchAmt: string (nullable = true)
 |-- PrevAmt: string (nullable = true)
 |-- PayMethod: string (nullable = true)



In [11]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

In [12]:
# using lambda function to convert date col to datetype
# changing datatypes of columns
func =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())
df2 = df1.withColumn("Election", df1["Election"].cast("Integer")) \
  .withColumn("OFFICECD", df1["OFFICECD"].cast("Integer")) \
  .withColumn("CandidateID", df1["CandidateID"].cast("Integer")) \
  .withColumn("ZipCode", df1["ZipCode"].cast("Integer")) \
  .withColumn("Amount", df1["Amount"].cast("Float")) \
  .withColumn("MatchAmt", df1["MatchAmt"].cast("Float")) \
  .withColumn("PrevAmt", df1["PrevAmt"].cast("Float")) \
  .withColumn('Date', func(col('Date')))
df2.printSchema()

AnalysisException: ignored

In [ ]:
#Change vaule name in ContributerType
from pyspark.sql.functions import regexp_replace
df3 = df2.withColumn('ContributerType', regexp_replace('ContributerType', 'CAN', 'Candidate')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'CORP', 'Corporation')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'EMPO', 'Labor Union')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'FAM', 'Candidate Family')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'IND', 'Individual')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PCOMZ', 'Party Committee')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'ORG', 'Orgainization')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'LLC', 'Limited Liability Company')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PCOMP', 'Political Action Committee')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'SPO', 'Spouse')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'OTHR', 'Other')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PCOMC', 'Candidate Committee')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PART', 'Individual')) \


In [ ]:
#Change Payment Method Name
df4=df3.withColumn('PayMethod', regexp_replace('PayMethod','0','Unknown')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','1','Cash')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','2','Check')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','3','Other')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','4','Credit Card')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','5','Money Order')) \

In [ ]:
#Change Borough name
from pyspark.sql.functions import regexp_replace
df5 = df4.withColumn('BoroughName', regexp_replace('BoroughName', 'K', 'Brooklyn')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'M', 'Manhattan')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'Q', 'Queens')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'S', 'Staten Island')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'X', 'Bronx')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'Z', 'Other')) \


In [ ]:
#Call only Mayor (1) and particaptes (P) within Dataframe
df6 = df5.filter((df5.OFFICECD=="1") & (df5.CANCLASS=="P"))
df6.show()

In [ ]:
#Check the above dataframe to make sure only unique values between the OFFICECD is 1 
df6.select('OFFICECD').distinct().show()

In [ ]:
#Drop OFFICECD and CANCLASS
list2 = ['OFFICECD', 'CANCLASS'] 
df7 = df6.drop(*list2)
df7.show()

In [ ]:
#Filter Dataframe removing the null CandidateID values
df8 = df7.filter(df7.CandidateID.isNotNull())

In [ ]:
# ADD AMNT and MATCHAMNT PrevAmnt
from pyspark.sql.functions import col
clean_indiv_201705_df = df8.withColumn("TotalAmount", col("Amount")+col("MatchAmt")+col('PrevAmt'))
clean_indiv_2005_df.show()

In [ ]:
#Export to Clean CSV
clean_indiv_2005_df.toPandas().to_csv("Clean_Indivdual_2005.csv", header=True)

## **2005 Committee Contributions**

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/Committee+Contributions+2005+(X).csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Committee+Contributions+2005+(X).csv"), sep=",", header=True)
df.show()

In [ ]:
# Remove multiple columns
list = ['COMMITTEE', 'FILING', 'SCHEDULE', 'PAGENO', 'SEQUENCENO','REFNO', 'STRNO', 'STRNAME', 'APARTMENT', 'OCCUPATION', 'EMPNAME', 'EMPSTRNO', 'EMPSTRNAME',
                'EMPCITY','REFUNDDATE','INTERMNO', 'INTERMNAME', 'INTSTRNO', 'INTSTRNM', 'INTSTRNM', 'INTAPTNO',
                'INTCITY', 'INTST','INTZIP', 'INTEMPNAME', 'INTEMPSTNO', 'INTEMPSTNM', 'INTEMPCITY', 'INTEMPST', 'INTOCCUPA' ,'PURPOSECD', 'EXEMPTCD','ADJTYPECD', 'RR_IND', 'SEG_IND','INT_C_CODE'] 
df = df.drop(*list)
df.show()

In [ ]:
# check data types
df.dtypes

In [ ]:
#Change column names
df1 = df \
.withColumnRenamed("ELECTION", "Election") \
.withColumnRenamed("RECIPID", "CandidateID") \
.withColumnRenamed("RECIPNAME", "CandidateName") \
.withColumnRenamed("DATE", "Date") \
.withColumnRenamed("NAME", "Name") \
.withColumnRenamed("C_CODE", "ContributerType") \
.withColumnRenamed("BOROUGHCD", "BoroughName") \
.withColumnRenamed("CITY", "City") \
.withColumnRenamed("STATE", "State") \
.withColumnRenamed("ZIP", "ZipCode") \
.withColumnRenamed("EMPSTATE", "ContributionState") \
.withColumnRenamed("AMNT", "Amount") \
.withColumnRenamed("MATCHAMNT", "MatchAmt") \
.withColumnRenamed("PREVAMNT", "PrevAmt") \
.withColumnRenamed("PAY_METHOD", "PayMethod") 

df1.printSchema()

In [ ]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

In [ ]:
# using lambda function to convert date col to datetype
# changing datatypes of columns
func =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())
df2 = df1.withColumn("Election",df1["Election"].cast("Integer")) \
  .withColumn("OFFICECD", df1["OFFICECD"].cast("Integer")) \
  .withColumn("CandidateID", df1["CandidateID"].cast("Integer")) \
  .withColumn("ZipCode", df1["ZipCode"].cast("Integer")) \
  .withColumn("Amount", df1["Amount"].cast("Float")) \
  .withColumn("MatchAmt", df1["MatchAmt"].cast("Float")) \
  .withColumn("PrevAmt", df1["PrevAmt"].cast("Float")) \
  .withColumn('Date', func(col('Date')))
df2.printSchema()

In [ ]:
#Change vaule name in ContributerType
from pyspark.sql.functions import regexp_replace
df3 = df2.withColumn('ContributerType', regexp_replace('ContributerType', 'CAN', 'Candidate')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'CORP', 'Corporation')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'EMPO', 'Labor Union')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'FAM', 'Candidate Family')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'IND', 'Individual')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PCOMZ', 'Party Committee')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'ORG', 'Orgainization')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'LLC', 'Limited Liability Company')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PCOMP', 'Political Action Committee')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'SPO', 'Spouse')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'OTHR', 'Other')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PCOMC', 'Candidate Committee')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PART', 'Individual')) \


In [ ]:
#Change Payment Method Name
df4=df3.withColumn('PayMethod', regexp_replace('PayMethod','0','Unknown')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','1','Cash')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','2','Check')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','3','Other')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','4','Credit Card')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','5','Money Order')) \

In [ ]:
#Change Borough name
from pyspark.sql.functions import regexp_replace
df5 = df4.withColumn('BoroughName', regexp_replace('BoroughName', 'K', 'Brooklyn')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'M', 'Manhattan')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'Q', 'Queens')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'S', 'Staten Island')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'X', 'Bronx')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'Z', 'Other')) \


In [ ]:
#Call only Mayor (1) and particaptes (P) within Dataframe
df6 = df5.filter((df5.OFFICECD=="1") & (df5.CANCLASS=="P"))
df6.show()

In [ ]:
#Check the above dataframe to make sure only unique values between the OFFICECD is 1 
df6.select('OFFICECD').distinct().show()

In [ ]:
#Drop OFFICECD and CANCLASS
list2 = ['OFFICECD', 'CANCLASS'] 
df7 = df6.drop(*list2)
df7.show()

In [ ]:
#Filter Dataframe removing the null CandidateID values
df8 = df7.filter(df7.CandidateID.isNotNull())

In [ ]:
# ADD AMNT and MATCHAMNT PrevAmnt
from pyspark.sql.functions import col
clean_comm_2005_df = df8.withColumn("TotalAmount", col("Amount")+col("MatchAmt")+col('PrevAmt'))
clean_comm_2005_df.show()

In [ ]:
#Export to new CSV
clean_comm_2005_df.toPandas().to_csv("Clean_Committee_2005.csv", header=True)

In [ ]:
# Store environmental variable
from getpass import getpass
#Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-final.cjdbapst3wjf.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": "******",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to active_user table in RDS
#clean_indiv_2005_df.write.jdbc(url=jdbc_url, table='individual_2005', mode=mode, properties=config)

In [ ]:
# Write DataFrame to active_user table in RDS
#clean_comm_2005_df.write.jdbc(url=jdbc_url, table='committee_2005', mode=mode, properties=config)